In [1]:
import re
import numpy as np
import cv2
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.layers import Input, Lambda, Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from keras.models import Sequential, Model
from keras.optimizers import RMSprop

In [2]:
# 图像对识别

import os
imglst=os.listdir(r"G:\Work\file\MNIST\data_a")

In [3]:
nums = dict(zip([str(i) for i in range(10)],[0]*10))
for name in imglst:
    nums[name[0]] += 1

In [35]:
total_sample_size = 10000

root_path = r'G:\Work\file\MNIST\data_a'

def get_data(total_sample_size):
    #read the image
    image = cv2.imread(root_path+"\\1_1.png", 0)
    # image = cv2.resize(image,(56,46),interpolation=cv2.INTER_CUBIC)
    #get the new size
    dim1 = image.shape[0]
    dim2 = image.shape[1]

    count = 0
    
    #initialize the numpy array with the shape of [total_sample, no_of_pairs, dim1, dim2]
    x_geuine_pair = np.zeros([total_sample_size, 2, dim1, dim2, 1])  # 2 is for pairs
    y_genuine = np.zeros([total_sample_size, 1])
    
    for i in range(10):
        for j in range(int(total_sample_size/10)):
            ind1 = 0
            ind2 = 0
            
            #read images from same directory (genuine pair)
            while ind1 == ind2:
                ind1 = np.random.randint(nums[str(i)])
                ind2 = np.random.randint(nums[str(i)])
            
            # read the two images
            img1 = cv2.imread(root_path+"\{}_{}.png".format(i, ind1), 0)
            img2 = cv2.imread(root_path+"\{}_{}.png".format(i, ind2), 0)
            # img1 = cv2.resize(img1,(56,46),interpolation=cv2.INTER_CUBIC)
            # img2 = cv2.resize(img2,(56,46),interpolation=cv2.INTER_CUBIC)
            
            #store the images to the initialized numpy array
            x_geuine_pair[count, 0, :, :, 0] = img1
            x_geuine_pair[count, 1, :, :, 0] = img2
            
            #as we are drawing images from the same directory we assign label as 1. (genuine pair)
            y_genuine[count] = 1
            count += 1

    count = 0
    x_imposite_pair = np.zeros([total_sample_size, 2, dim1, dim2, 1])
    y_imposite = np.zeros([total_sample_size, 1])
    
    for i in range(int(total_sample_size/10)):
        for j in range(10):
            
            #read images from different directory (imposite pair)
            while True:
                ind1 = np.random.randint(10)
                ind2 = np.random.randint(10)
                if ind1 != ind2:
                    break
                    
            img1 = cv2.imread(root_path+'\{}_{}.png'.format(ind1, j), 0)
            img2 = cv2.imread(root_path+'\{}_{}.png'.format(ind2, j), 0)
            # img1 = cv2.resize(img1,(56,46),interpolation=cv2.INTER_CUBIC)
            # img2 = cv2.resize(img2,(56,46),interpolation=cv2.INTER_CUBIC)


            x_imposite_pair[count, 0, :, :, 0] = img1
            x_imposite_pair[count, 1, :, :, 0] = img2
            #as we are drawing images from the different directory we assign label as 0. (imposite pair)
            y_imposite[count] = 0
            count += 1
            
    #now, concatenate, genuine pairs and imposite pair to get the whole data
    X = np.concatenate([x_geuine_pair, x_imposite_pair], axis=0)/255
    Y = np.concatenate([y_genuine, y_imposite], axis=0)

    return X, Y

In [36]:
X, Y = get_data(total_sample_size)
print("X shape:", X.shape)
print("Y shape:", Y.shape)

X shape: (20000, 2, 28, 28, 1)
Y shape: (20000, 1)


In [66]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [67]:
def build_base_network(input_shape):
    
    seq = Sequential()
    kernel_size = 2
    
    #convolutional layer 1
    seq.add(Conv2D(4, kernel_size, kernel_size, input_shape=input_shape,  data_format='channels_last', padding='valid', activation='relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))  
    seq.add(Dropout(.25))
    
    #convolutional layer 2
    # seq.add(Conv2D(nb_filter[1], kernel_size, kernel_size, padding='valid',  data_format='channels_first', activation='relu'))
    # seq.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_first')) 
    # seq.add(Dropout(.25))

    #flatten 
    seq.add(Flatten())
    seq.add(Dense(128, activation='relu'))
    seq.add(Dropout(0.1))
    seq.add(Dense(50, activation='relu'))
    seq.summary()

    return seq

In [68]:
input_dim = x_train.shape[2:]
input_a = Input(shape=input_dim)
input_b = Input(shape=input_dim)

base_network = build_base_network(input_dim)
feat_vecs_a = base_network(input_a)
feat_vecs_b = base_network(input_b)

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 14, 14, 4)         20        
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 7, 7, 4)           0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 7, 7, 4)           0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 196)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 128)               25216     
_________________________________________________________________
dropout_17 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 50)              

In [69]:
def euclidean_distance(vects):
    x, y = vects
    return K.sqrt(K.sum(K.square(x-y), axis=1, keepdims=True))

def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0],1)

In [70]:
distance = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([feat_vecs_a, feat_vecs_b])

In [71]:
epochs = 5
rms = RMSprop()

model = Model(inputs=[input_a, input_b], outputs=distance)

In [72]:
def contrastive_loss(y_true, y_pred):
    margin = 1
    return K.mean(y_true*K.square(y_pred)+(1-y_true)*K.square(K.maximum(margin-y_pred, 0)))

model.compile(loss=contrastive_loss, optimizer=rms)

In [73]:
img_1 = x_train[:, 0]
img_2 = x_train[:, 1]

model.fit([img_1, img_2], y_train, validation_split=0.2, batch_size=128, verbose=2, epochs=epochs)

Epoch 1/5
100/100 - 1s - loss: 0.1882 - val_loss: 0.1256
Epoch 2/5
100/100 - 1s - loss: 0.1345 - val_loss: 0.0912
Epoch 3/5
100/100 - 1s - loss: 0.1090 - val_loss: 0.0721
Epoch 4/5
100/100 - 1s - loss: 0.0920 - val_loss: 0.0562
Epoch 5/5
100/100 - 1s - loss: 0.0780 - val_loss: 0.0433


In [74]:
pred = model.predict([x_test[:, 0], x_test[:, 1]])
def compute_accuracy(pred, labels):
    return labels[pred.ravel() < 0.5].mean()

compute_accuracy(pred, y_test)

0.9921491658488715